<a href="https://colab.research.google.com/github/AhmetSarici/DataScience/blob/ML/CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CatBoost

Kategorik değişkenler ile otomatik olarak mücadele edebilen, hızlı bir gbm türüdür.

Hızlı ve ölçeklenebilir GPU desteği.


In [ ]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
from catboost import CatBoostRegressor

In [ ]:
catb = CatBoostRegressor()
catb_model = catb.fit(X_train, y_train)

Learning rate set to 0.031674
0:	learn: 437.6430699	total: 49.1ms	remaining: 49.1s
1:	learn: 431.3923642	total: 51.3ms	remaining: 25.6s
2:	learn: 424.8820360	total: 53.3ms	remaining: 17.7s
3:	learn: 418.2514904	total: 55.4ms	remaining: 13.8s
4:	learn: 412.6394021	total: 57.5ms	remaining: 11.4s
5:	learn: 406.6247020	total: 59.5ms	remaining: 9.86s
6:	learn: 400.5321206	total: 61.6ms	remaining: 8.74s
7:	learn: 394.6683437	total: 63.8ms	remaining: 7.91s
8:	learn: 388.2496484	total: 65.9ms	remaining: 7.26s
9:	learn: 382.9448842	total: 68ms	remaining: 6.73s
10:	learn: 377.2600080	total: 70ms	remaining: 6.29s
11:	learn: 372.4829606	total: 72ms	remaining: 5.93s
12:	learn: 366.6823437	total: 74.2ms	remaining: 5.63s
13:	learn: 362.6076230	total: 77.8ms	remaining: 5.48s
14:	learn: 358.0107745	total: 82.1ms	remaining: 5.39s
15:	learn: 353.2802665	total: 84.3ms	remaining: 5.19s
16:	learn: 348.5646265	total: 86.4ms	remaining: 5s
17:	learn: 343.6407912	total: 88.6ms	remaining: 4.83s
18:	learn: 339.23

## Tahmin

In [ ]:
y_pred = catb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

351.194631344607

## Model Tuning

In [ ]:
catb_grid = {
    'iterations': [200,500,1000,2000],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'depth': [3,4,5,6,7,8] }

In [ ]:
catb = CatBoostRegressor()
catb_cv_model = GridSearchCV(catb, catb_grid, cv=5, n_jobs = -1, verbose = 2)

In [ ]:
catb_cv_model.fit(X_train, y_train)

In [ ]:
catb_cv_model.best_params_

{'depth': 5, 'iterations': 1000, 'learning_rate': 0.1}

In [ ]:
catb_tuned = CatBoostRegressor(iterations = 200, 
                               learning_rate = 0.01, 
                               depth = 8)

catb_tuned = catb_tuned.fit(X_train,y_train)

In [ ]:
y_pred = catb_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

369.6970696250705

In [ ]:
catb_tuned2 = CatBoostRegressor(iterations = 500, 
                               learning_rate = 0.02, 
                               depth = 4)

catb_tuned2 = catb_tuned2.fit(X_train,y_train)

In [ ]:
y_pred2 = catb_tuned2.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred2))

347.360088063874